# Using Portlogit to estimate a portfolio choice model with budget constraint

In this notebook, we estimate the portfolio choice model with budget constraint using toy data for purpose.

## The portfolio choice model

### Formalisation

The portfolio choice model is an extension of the Random Utility Maximisation (RUM) model (McFadden, 1974) to accomodate composite (portfolio) choices of non mutually-exclusive alternatives. The model is further expanded by Bahamonde-Birke and Mouter (2019) to accomodate budget constraints.

In this model, decision-makers seek to maximise their utility from combinations of alternatives (i.e., goods, policies, etc.). Specifically, the utility of a combination of alternatives $p$ by individual $n$ is formalised as:

$$ U_{np} = \sum_{j=1}^J y_{nj} \cdot V_{nj} + \delta_0 \cdot (B - \sum_{j=1}^J y_{nj} \cdot c_{nj}) + \varepsilon_{np},$$

where $y_{nj}$ is a binary variable equal to one if alternative $j$ is selected, and zero otherwise, $V_{nj}$ is the utility associated with alternative $j$, $B$ is the budget constraint (if any), $c_{nj}$ are the costs of each alternative (if any), $\delta_0$ is a parameter to be estimated and $\varepsilon_{np}$ is a stochastic error term. The observable utility of each alternative is linear in parameters. Thus, $V_{nj} = \delta_j + \beta'X_j$, where $\delta_j$ and $\beta$ are parameters to be estimated.

### Economic interpretation of the estimated parameters

The parameters $\delta_0$, $\delta_j$ and $\beta$ have economic interpretation. The parameter $\delta_0$ quantifies the preferences for spending the available budget. If $\delta_0$ is positive, then respondents prefer to spend more budget, on average, and viceversa if $\delta_0$ is negative. The parameters $\delta_j$ are alternative-specific constants that detail how the utility changes when alternative $j$ is selected. If $\delta_j$ is positive, then, on average, including alternative $j$ is preferred than not including it, _ceteris paribus_. $\beta$ is a vector of parameters associated with the attributes of the alternatives. If a parameter included in $\beta$ is positive, it means that, on average, increases of the correspondent attribute are preferred by individuals. Conversely, if $\beta$ is negative, then increases of the attribute generate dissatisfaction, on average.

## Estimating a portfolio choice model with PortChoice

### Step 1: Load modules and dataset

The following lines load the modules required to open the data and estimate the portfolio choice model:

In [11]:
import pandas as pd
import numpy as np
from portchoice.models import PortLogit

The dataset is contained in the folder `data` and it is named `toy_data_5.csv`. This dataset contains 10,000 responses to a fictional portfolio choice experiment (i.e., a PVE without budget constraint). Each respondent faced **four alternatives** characterised by **three attributes each alternative**. Additionally, each alternative has a cost of resources, and respondents cannot surpass a specific budget constraint of 1.82 euros.

The dependent variables are:

- `Choice_1-4`: Binary choice indicator per alternative, equal to one if the correspondent alternative was chosen, and zero otherwise.

The independent variables are:

- `X_1_1-4`: Attribute 1, created from a uniform distribution
- `X_2_1-4`: Attribute 2, created from a uniform distribution
- `X_3_1-4`: Attribute 3, created from a uniform distribution

The cost and constraint variables are

- `Cost_1-4`: The cost variables
- `Budget`: The budget constraint per individual.

We use `Pandas` to open the dataset in a data frame and preview the data

In [12]:
# Load files
data = pd.read_csv('../data/toy_data_5.csv',index_col=None)
data

,Unnamed: 0,Choice_1,Choice_2,Choice_3,Choice_4,X_1_1,X_2_1,X_3_1,X_1_2,X_2_2,...,X_2_3,X_3_3,X_1_4,X_2_4,X_3_4,Cost_1,Cost_2,Cost_3,Cost_4,Budget
0,0,1.0,0.0,1.0,1.0,0.700437,0.844187,0.676514,0.727858,0.951458,...,0.048813,0.099929,0.508066,0.200248,0.744154,0.408396,0.382224,0.803101,0.305588,1.82012
1,1,1.0,0.0,1.0,1.0,0.192892,0.700845,0.293228,0.774479,0.005109,...,0.247668,0.023236,0.727321,0.340035,0.197503,0.131546,0.127560,0.158345,0.211066,1.82012
2,2,0.0,1.0,1.0,1.0,0.909180,0.978347,0.532803,0.259132,0.583813,...,0.626405,0.818874,0.547345,0.416712,0.743047,0.623518,0.051760,0.728043,0.825899,1.82012
3,3,1.0,1.0,0.0,1.0,0.369596,0.075167,0.775193,0.219409,0.079342,...,0.828465,0.191369,0.270409,0.561034,0.902380,0.513754,0.071600,0.670793,0.839230,1.82012
4,4,0.0,1.0,1.0,1.0,0.851788,0.418082,0.393476,0.016221,0.299213,...,0.786137,0.771387,0.420055,0.776025,0.464308,0.562920,0.456286,0.436227,0.106833,1.82012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0.0,1.0,1.0,1.0,0.355965,0.883178,0.166369,0.605301,0.515490,...,0.755789,0.540213,0.619993,0.523788,0.976918,0.487538,0.713338,0.688247,0.278711,1.82012
9996,9996,0.0,1.0,1.0,0.0,0.940168,0.441731,0.451852,0.287348,0.896920,...,0.601479,0.932629,0.011957,0.806935,0.384588,0.810090,0.082316,0.898550,0.854034,1.82012
9997,9997,0.0,1.0,0.0,1.0,0.451202,0.020886,0.040896,0.028147,0.153752,...,0.962383,0.055374,0.594689,0.392153,0.575639,0.728140,0.086375,0.108417,0.608334,1.82012
9998,9998,1.0,0.0,1.0,0.0,0.472081,0.804692,0.899335,0.211611,0.989515,...,0.954349,0.667504,0.181257,0.994077,0.022506,0.768954,0.757814,0.665002,0.573090,1.82012


### Step 2: create arrays to estimate the portfolio choice model

In this step, we create the arrays of the dependent and independent variables for the estimation. We create a new dataframe, called `y`, which contains the dependent variables:

In [13]:
y = data[['Choice_1','Choice_2','Choice_3','Choice_4']]
y

,Choice_1,Choice_2,Choice_3,Choice_4
0,1.0,0.0,1.0,1.0
1,1.0,0.0,1.0,1.0
2,0.0,1.0,1.0,1.0
3,1.0,1.0,0.0,1.0
4,0.0,1.0,1.0,1.0
...,...,...,...,...
9995,0.0,1.0,1.0,1.0
9996,0.0,1.0,1.0,0.0
9997,0.0,1.0,0.0,1.0
9998,1.0,0.0,1.0,0.0


Then, we create the array of independent variables. This array must have the following requirements:

- The number of attributes per alternative must be the same
- The variables are ordered by attribute and by alternative.

In our case, the first independent variables correspond to the attributes of alternative 1, then for alternative 2, and so on until alternative 4:

In [14]:
X = data[
    ['X_1_1','X_2_1','X_3_1',
     'X_1_2','X_2_2','X_3_2',
     'X_1_3','X_2_3','X_3_3',
     'X_1_4','X_2_4','X_3_4',]]

X

,X_1_1,X_2_1,X_3_1,X_1_2,X_2_2,X_3_2,X_1_3,X_2_3,X_3_3,X_1_4,X_2_4,X_3_4
0,0.700437,0.844187,0.676514,0.727858,0.951458,0.012703,0.413588,0.048813,0.099929,0.508066,0.200248,0.744154
1,0.192892,0.700845,0.293228,0.774479,0.005109,0.112858,0.110954,0.247668,0.023236,0.727321,0.340035,0.197503
2,0.909180,0.978347,0.532803,0.259132,0.583813,0.325691,0.888899,0.626405,0.818874,0.547345,0.416712,0.743047
3,0.369596,0.075167,0.775193,0.219409,0.079342,0.486781,0.153674,0.828465,0.191369,0.270409,0.561034,0.902380
4,0.851788,0.418082,0.393476,0.016221,0.299213,0.353778,0.893503,0.786137,0.771387,0.420055,0.776025,0.464308
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.355965,0.883178,0.166369,0.605301,0.515490,0.744056,0.475863,0.755789,0.540213,0.619993,0.523788,0.976918
9996,0.940168,0.441731,0.451852,0.287348,0.896920,0.851084,0.812522,0.601479,0.932629,0.011957,0.806935,0.384588
9997,0.451202,0.020886,0.040896,0.028147,0.153752,0.767910,0.962120,0.962383,0.055374,0.594689,0.392153,0.575639
9998,0.472081,0.804692,0.899335,0.211611,0.989515,0.353981,0.938232,0.954349,0.667504,0.181257,0.994077,0.022506


Finally, we create the cost and buget arrays:

In [15]:
costs = data[['Cost_1','Cost_2','Cost_3','Cost_4']]
budget = data['Budget']

costs

,Cost_1,Cost_2,Cost_3,Cost_4
0,0.408396,0.382224,0.803101,0.305588
1,0.131546,0.127560,0.158345,0.211066
2,0.623518,0.051760,0.728043,0.825899
3,0.513754,0.071600,0.670793,0.839230
4,0.562920,0.456286,0.436227,0.106833
...,...,...,...,...
9995,0.487538,0.713338,0.688247,0.278711
9996,0.810090,0.082316,0.898550,0.854034
9997,0.728140,0.086375,0.108417,0.608334
9998,0.768954,0.757814,0.665002,0.573090


In [16]:
budget

0       1.82012
1       1.82012
2       1.82012
3       1.82012
4       1.82012
         ...   
9995    1.82012
9996    1.82012
9997    1.82012
9998    1.82012
9999    1.82012
Name: Budget, Length: 10000, dtype: float64

### Step 3: Create the portfolio choice model object and estimate

Now, we create the portfolio choice model object (`PortLogit`). The parameters of this object are:

- `Y`: A data frame with choices of each alternative for each respondent.
- `X`: (optional) A data frame with the alternative-specific variables (e.g., attributes)
- `Z`: (optional)A data frame with the individual-specific variables
- `C`: (optional)A data frame with the costs of each individual alternative for each respondent
- `B`: (optional)Resource constraint
- `B_init`: (optional)Initial level of consumed resources, by default 0.
- `interactions`: (optional) List of alternative-interactions. Each element is a list that marks the alternatives that interact, from 0 to J-1, where J is the number of alternatives.
-  `base_combinations`: (optional) Array with initial set of combinations. Can be used to discard unfeasible combinations upfront. If no list is provided, PortChoice will construct a set of all possible combinations from a full-factorial design.
- `mutually_exclusive`: (optional) List of mutually-exclusive alternatives. Each element of the list is a `numpy` array of two elements that detail the two mutually-exclusive alternatives.

Notice that, except for `Y`, all other parameters of `PortLogit` are optional. This is because you can always estimate a portfolio choice model with only constants per alternative. In this case, we estimate a portfolio choice model with attributes per alternative (`X`). The following lines create the model object:

In [17]:
# Create model
obj = PortLogit(Y=y,X=X,C=costs,B=budget)

To estimate the model, we use the method `estimate()`, which takes the following parameters for this example:

- `startv`: Starting values for the maximum-likelihood estimation routine.
- `asc`: An array of length `n_alternatives`, in which each element can be either equal to one if the ASC of the corresponding alternative is estimated, and zero otherwise.
- `delta_0`: If None and `C` exists, then the parameter of the marginal utility of non-spent resources is estimated. If `delta_0` is a float, then the parameter is fixed to the value of `delta_0`.
- `method`: The optimisation method for the MLE routine. Available options are either the built-in BFGS minimiser (`bfgsmin`) or a method available for `scipy.minimize.optimize`, by detault `bfgsmin`

The estimation routine returns the following information:

- `ll`: Log-likelihood function at the optimum
- `coef`: Estimated parameters at the optimum
- `se`: Standard errors of `coef`. If `hess = False` and `method` is not 'bfgsmin' then `se = 0.`, else if method is 'bfgsmin', it returns the standard errors computed from the Hessian approximation.
- `hessian`: Finite-difference Hessian. If `hess = False` and `method` is not 'bfgsmin' then `hessian = 0.`, else if method is 'bfgsmin', it returns the Hessian approximation.
- `diff_time`: Estimation time in seconds.

We will estimate a portfolio choice model with alternative-specific constants. The following lines provide the detailed code:

In [18]:
# Estimate
asc = np.ones(4).astype(int)
startv = np.zeros(asc.sum() + 3 + 1)
fval, coef, se, hessian, diff_time = obj.estimate(startv,delta_0=None,asc=asc,hess=False)

Initial F-value: 26417.29
Iter No. 1: F-value: 21916.29 / Step size: 0.000244 / G-norm: 88833708.27277
Iter No. 2: F-value: 20792.26 / Step size: 0.003906 / G-norm: 14076200.434776
Iter No. 3: F-value: 20533.44 / Step size: 0.003906 / G-norm: 5126280.377711
Iter No. 4: F-value: 20284.14 / Step size: 0.007812 / G-norm: 89494.291266
Iter No. 5: F-value: 20260.93 / Step size: 0.015625 / G-norm: 144230.268583
Iter No. 6: F-value: 20118.67 / Step size: 0.015625 / G-norm: 49887.755519
Iter No. 7: F-value: 20033.82 / Step size: 0.0625 / G-norm: 18967.852778
Iter No. 8: F-value: 18243.79 / Step size: 0.015625 / G-norm: 253011.074403
Iter No. 9: F-value: 13824.76 / Step size: 0.03125 / G-norm: 611140.071758
Iter No. 10: F-value: 9821.71 / Step size: 1 / G-norm: 10948.095556
Iter No. 11: F-value: 9263.84 / Step size: 1 / G-norm: 871.883646
Iter No. 12: F-value: 9194.63 / Step size: 1 / G-norm: 144.993812
Iter No. 13: F-value: 9175.2 / Step size: 1 / G-norm: 21.550058
Iter No. 14: F-value: 9119.4

In [21]:
coef

array([ 1.05596807,  2.13024847,  3.07158777,  4.10230044, -5.98199474,
       -3.04414405,  9.98319226,  1.0418734 ])

In [22]:
obj.coef

array([ 1.05596807,  2.13024847,  3.07158777,  4.10230044, -5.98199474,
       -3.04414405,  9.98319226,  1.0418734 ])

Now we visualise the results:

In [19]:
# Construct results matrix
results = pd.DataFrame(np.c_[coef,se,coef/se],columns=['Estimate','Std.Err.','T-stat'],index=['ASC_1','ASC_2','ASC_3','ASC_4'] + ['X_1','X_2','X_3'] + ['Cost'])
print(results)

       Estimate  Std.Err.     T-stat
ASC_1  1.055968  0.071535  14.761527
ASC_2  2.130248  0.076597  27.811081
ASC_3  3.071588  0.079557  38.608818
ASC_4  4.102300  0.087848  46.697533
X_1   -5.981995  0.094420 -63.354934
X_2   -3.044144  0.070109 -43.420026
X_3    9.983192  0.131201  76.090742
Cost   1.041873  0.081581  12.771025


### Step 4: the optimal portfolio

The optimal portfolio is the ranking of combinations of alternatives, sorted by their expected utility. The expected utility is computed using the estimated coefficients of the portfolio choice model for a given set of attributes, and using numerical simulation of the error terms.

To compute the optimal portfolio, we use the method `optimal_portfolio()` **after the estimation**. This method takes the following parameters:

- `X`: (optional) Series of alternative-specific variables
- `Z`: (optional) Data frame with individual-specific variables
- `C`: (optional) Series with individual costs per alternative
- `B`: (optional) Resource constraint
- `sims`: (optional) Number of Extreme Value random draws, by default 1000

Since our model was estimated with attributes, we use the mean values of such attributes for the optimal portfolio. The following lines compute and print the optimal portfolio

In [20]:
portfolio = obj.optimal_portfolio(X = X.mean(),C=costs.mean(),B=budget.mean(),sims=10000)
print(portfolio)

    Alt_1  Alt_2  Alt_3  Alt_4         EU  Totalcosts
0     0.0    1.0    1.0    1.0  11.803780    1.513724
1     1.0    0.0    1.0    1.0  10.612660    1.513359
2     1.0    1.0    0.0    1.0   9.696277    1.507068
3     0.0    0.0    1.0    1.0   9.644019    1.013207
4     0.0    1.0    0.0    1.0   8.724863    1.006916
5     1.0    1.0    1.0    0.0   8.636744    1.507478
6     0.0    1.0    1.0    0.0   7.659737    1.007326
7     1.0    0.0    0.0    1.0   7.556483    1.006551
8     0.0    0.0    0.0    1.0   6.588259    0.506398
9     1.0    0.0    1.0    0.0   6.510750    1.006961
10    1.0    1.0    0.0    0.0   5.582491    1.000670
11    0.0    0.0    1.0    0.0   5.519998    0.506808
12    0.0    1.0    0.0    0.0   4.620465    0.500517
13    1.0    0.0    0.0    0.0   3.441296    0.500152
14    0.0    0.0    0.0    0.0   2.468896    0.000000
15    NaN    NaN    NaN    NaN        NaN    2.013876
